In [5]:
from helpers import *
from interpolation import *
from solvers import *
import matplotlib.pyplot as plt
import os

class TestCase:
    def __init__(self, matrix: Matrix, filename: str):
        self.matrix = matrix
        self.filename = filename
        self.n = matrix.shape[0]
        self.x = []
        self.y = []
        for i in range(self.n):
            self.x.append(matrix[i][0])
            self.y.append(matrix[i][1])


files = [('chelm.txt', ' '), ('in_mountain.data', ','), ('stale.txt', ','), ('ulm_lugano.txt', ' '), ('tczew_starogard.txt', ' '), ('genoa_rapallo.txt', ' ')]
test_cases = [TestCase(load_data(file, separator), file) for file, separator in files]
    

In [6]:
# Generate base plots 
for test_case in test_cases:
    os.makedirs(f'../plots/{test_case.filename}', exist_ok=True)

    plt.figure(figsize=(15, 5))
    plt.plot(test_case.x, test_case.y, label='data points')
    plt.xlabel('Dystans')
    plt.ylabel('Wysokość')
    plt.grid()
    plt.title(f'Dane wejściowe - {test_case.filename} ({test_case.n} punktów)')
    plt.savefig(f'../plots/{test_case.filename}/input_data.png', bbox_inches='tight')
    plt.close()

In [7]:
# Interpolate using Lagrange and generate plots
tests = [(100, 512), (50, 512), (25, 512), (15, 512), (10, 512), (5, 512)]

def plot_interpolation(title: str, filename: str, x: list[float], y: list[float], selected_x: list[float], selected_y):
    plt.figure(figsize=(15, 5))
    plt.title(title)
    plt.semilogy(x, y, label='Interpolacja')
    plt.semilogy(test_case.x, test_case.y, label='Dane wejściowe')
    plt.semilogy(selected_x, selected_y, '.', label='Wybrane punkty')
    plt.xlabel('Dystans')
    plt.ylabel('Wysokość (log)')
    plt.legend()
    plt.grid()
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

for test_case in test_cases:
    print(f'Interpolating {test_case.filename}: ', end='')
    for points, nodes in tests:
        selected_x = test_case.x[::int(test_case.n/points)]
        selected_y = test_case.y[::int(test_case.n/points)]
        size = len(selected_x) # for cases when n is not multiple of points

        x = linspace(test_case.x[0], test_case.x[-1], nodes)
        y = lagrange_interpolation(x, selected_x, selected_y)

        plot_interpolation(f'Interpolacja Lagrange\'a - {test_case.filename} ({len(x)} punktów interpolacji, {size} punktów wejściowych)',
                        f'../plots/{test_case.filename}/lagrange_{size}_points.png', x, y, selected_x, selected_y)
        
        chebyshev_nodes = generate_chebyshev_nodes(test_case.x[0], test_case.x[-1], size)
        chebyshev_selected_x, chebyshev_selected_y = [], []
        zipped = list(zip(test_case.x, test_case.y))
        # find closest points to chebyshev nodes
        for node in chebyshev_nodes:
            closest = min(zipped, key=lambda p: abs(p[0] - node))
            chebyshev_selected_x.append(closest[0])
            chebyshev_selected_y.append(closest[1])
            zipped.remove(closest)

        x_chebyshev = linspace(test_case.x[0], test_case.x[-1], nodes)
        y_chebyshev = lagrange_interpolation(x_chebyshev, chebyshev_selected_x, chebyshev_selected_y)

        plot_interpolation(f'Interpolacja Lagrange\'a z węzłami Czebyszewa - {test_case.filename} ({len(x)} punktów interpolacji, {size} punktów wejściowych)',
                        f'../plots/{test_case.filename}/lagrange_{size}_points_chebyshev.png', x_chebyshev, y_chebyshev, chebyshev_selected_x, chebyshev_selected_y)

        print(f'{size} ', end='')
    print()

Interpolating chelm.txt: 103 52 26 16 11 6 
Interpolating in_mountain.data: 103 52 26 16 11 6 
Interpolating stale.txt: 103 52 26 16 11 6 
Interpolating ulm_lugano.txt: 103 52 26 16 11 6 
Interpolating tczew_starogard.txt: 103 52 26 16 11 6 
Interpolating genoa_rapallo.txt: 103 52 26 16 11 6 


In [8]:
# Interpolate using splines and generate plots
for test_case in test_cases:
    print(f'Interpolating {test_case.filename}: ', end='')
    for points, nodes in tests:
        selected_x = test_case.x[::int(test_case.n/points)]
        selected_y = test_case.y[::int(test_case.n/points)]
        size  = len(selected_x)

        x = linspace(test_case.x[0], test_case.x[-1], nodes)
        y = spline_interpolation(x, selected_x, selected_y)

        plot_interpolation(f'Interpolacja funkcjami sklejanymi - {test_case.filename} ({len(x)} punktów interpolacji, {size} punktów wejściowych)',
                        f'../plots/{test_case.filename}/spline_{size}_points.png', x, y, selected_x, selected_y)
        
        print(f'{size} ', end='')
    print()

Interpolating chelm.txt: 103 52 26 16 11 6 
Interpolating in_mountain.data: 103 52 26 16 11 6 
Interpolating stale.txt: 103 52 26 16 11 6 
Interpolating ulm_lugano.txt: 103 52 26 16 11 6 
Interpolating tczew_starogard.txt: 103 52 26 16 11 6 
Interpolating genoa_rapallo.txt: 103 52 26 16 11 6 
